In [ ]:
import numpy as np
import shutil
from scipy.interpolate import interp1d
import h5py

# function that will compute the effective density values to give to Rockstar 
def compute_effective_cosmology(
    a_snap,
    a_bg,
    rho_em,
    rho_lm,
    rho_rad,
    h0_true
):
  
    a_bg = np.asarray(a_bg)
    rho_em = np.asarray(rho_em)
    rho_lm = np.asarray(rho_lm)
    rho_rad = np.asarray(rho_rad)

    if np.any(np.diff(a_bg) <= 0):
        raise ValueError("a_bg must be strictly increasing.")

    #interpolators
    kind = "cubic"
    f_em  = interp1d(a_bg, rho_em,  kind=kind, fill_value="extrapolate")
    f_lm  = interp1d(a_bg, rho_lm,  kind=kind, fill_value="extrapolate")
    f_rad = interp1d(a_bg, rho_rad, kind=kind, fill_value="extrapolate")

    rho_em_snap  = float(f_em(a_snap))
    rho_lm_snap  = float(f_lm(a_snap))
    rho_rad_snap = float(f_rad(a_snap))

    rho_tot_snap = rho_em_snap + rho_lm_snap + rho_rad_snap

    Omegam_true_snap = (rho_em_snap + rho_lm_snap) / rho_tot_snap


    a = float(a_snap)
    y = Omegam_true_snap
    a3inv = a**-3

    Om0_eff = y / (a3inv - y * (a3inv - 1.0))
    Ol0_eff = 1.0 - Om0_eff


    f_tot = interp1d(a_bg, rho_em + rho_lm + rho_rad,
                     kind=kind, fill_value="extrapolate")

    rho_tot_today = float(f_tot(1.0)) 
    E_true_snap = np.sqrt(rho_tot_snap / rho_tot_today)
    h_eff = h0_true * E_true_snap

    return Om0_eff, Ol0_eff, h_eff, Omegam_true_snap



In [ ]:

gadget_header_dtype = np.dtype([
    ('npart',                ('<i4', 6)),   
    ('mass',                 ('<f8', 6)),   
    ('time',                 '<f8'),        
    ('redshift',             '<f8'),        
    ('flag_sfr',             '<i4'),        
    ('flag_feedback',        '<i4'),      
    ('npartTotal',           ('<i4', 6)),   
    ('flag_cooling',         '<i4'),        
    ('num_files',            '<i4'),        
    ('BoxSize',              '<f8'),       
    ('Omega0',               '<f8'),       
    ('OmegaLambda',          '<f8'),        
    ('HubbleParam',          '<f8'),       
    ('flag_stellarage',      '<i4'),       
    ('flag_metals',          '<i4'),        
    ('npartTotalHighWord',   ('<i4', 6)),   
    ('flag_entropy_instead_u','<i4'),       
    ('flag_doubleprecision', '<i4'),        
    ('flag_lpt_ics',         '<i4'),       
    ('lpt_scalingfactor',    '<f4'),       
    ('fill',                 '48b'),      
])




def _patch_header_struct(header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift):
    header['Omega0']      = Om0_eff
    header['OmegaLambda'] = Ol0_eff
    header['HubbleParam'] = h_eff
    if update_time_and_redshift:
        header['time']     = a_snap
        header['redshift'] = 1.0 / a_snap - 1.0
    return header


def patch_gadget_header_cosmology(
    infile,
    outfile,
    a_snap,
    Om0_eff,
    Ol0_eff,
    h_eff,
    update_time_and_redshift=True
):
   

    if infile != outfile:
        shutil.copy2(infile, outfile)

    with open(outfile, "r+b") as f:
        # Peek at the first 4 bytes to decide format
        first = np.fromfile(f, dtype='<i4', count=1)
        if first.size != 1:
            raise IOError("Could not read first 4 bytes from snapshot.")

        first_val = int(first[0])

    
        # -----------------------------
        if first_val == 256:
            header_arr = np.fromfile(f, dtype=gadget_header_dtype, count=1)
            if header_arr.size != 1:
                raise IOError("Could not read GADGET header (format 1).")
            header = header_arr[0]

            header = _patch_header_struct(
                header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift
            )

            f.seek(4)  
            np.array([header], dtype=gadget_header_dtype).tofile(f)
            return

        # -----------------------------

        elif first_val == 8:
            blockname_bytes = f.read(4)
            if len(blockname_bytes) != 4:
                raise IOError("Could not read blockname for format 2 snapshot.")
            blockname = blockname_bytes.decode('ascii', errors='ignore')

            skip_len_arr = np.fromfile(f, dtype='<i4', count=1)
            if skip_len_arr.size != 1:
                raise IOError("Could not read skip length for format 2 snapshot.")
            skip_len = int(skip_len_arr[0])

            end_block_arr = np.fromfile(f, dtype='<i4', count=1)
            if end_block_arr.size != 1:
                raise IOError("Could not read end-of-name-block marker.")
            if int(end_block_arr[0]) != 8:
                raise ValueError(
                    f"Unexpected end-of-name-block marker: {end_block_arr[0]} (expected 8)."
                )

            if blockname.strip().upper() not in ("HEAD", "HEADER"):
                raise ValueError(
                    f"First block name is '{blockname}', not 'HEAD'; "
                    "this patcher assumes HEAD is the first block."
                )

            data_blocksize_arr = np.fromfile(f, dtype='<i4', count=1)
            if data_blocksize_arr.size != 1:
                raise IOError("Could not read header data block size.")
            data_blocksize = int(data_blocksize_arr[0])

            data_start_pos = f.tell()
            raw = bytearray(f.read(data_blocksize))
            if len(raw) != data_blocksize:
                raise IOError("Could not read full header data block.")

            needed = gadget_header_dtype.itemsize
            if data_blocksize < needed:
                raise ValueError(
                    f"Header data block ({data_blocksize} bytes) smaller than expected "
                    f"GADGET header size ({needed} bytes)."
                )

            header = np.frombuffer(raw[:needed], dtype=gadget_header_dtype, count=1)[0]
            header = _patch_header_struct(
                header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift
            )

            header_bytes = np.array([header], dtype=gadget_header_dtype).tobytes()
            raw[:needed] = header_bytes

            f.seek(data_start_pos)
            f.write(raw)

          
            return

        else:
            raise ValueError(
                f"Unrecognized first int in snapshot: {first_val}. "
                "Expected 256 (format 1) or 8 (format 2)."
            )


In [ ]:
def patch_snapshot(
    snapshot_in,
    snapshot_out,
    a_snap,
    a_bg,
    rho_em,
    rho_lm,
    rho_rad,
    h0_true
):
    Om0_eff, Ol0_eff, h_eff, Om_true = compute_effective_cosmology(
        a_snap, a_bg, rho_em, rho_lm, rho_rad, h0_true
    )
    print(f"a_snap={a_snap:.4f}, Ωm_true={Om_true:.5f}, "
          f"Ωm0_eff={Om0_eff:.5f}, ΩΛ0_eff={Ol0_eff:.5f}, h_eff={h_eff:.5f}")

    patch_gadget_header_cosmology(
        snapshot_in,
        snapshot_out,
        a_snap,
        Om0_eff,
        Ol0_eff,
        h_eff,
        update_time_and_redshift=True,  
    )

In [ ]:
# update file here with your class_processed.hdf5 file from CONCEPT
file = '/mnt/c/Users/doman/OneDrive/Desktop/class_processed.hdf5'


with h5py.File(file, "r") as f:
    bg = f["background"]

    t = bg["t"][:]   # cosmic time [Gyr by default]
    a_bg = bg["a"][:]   # scale factor
    z = bg["z"][:]   # redshift
    
    print('Available background quantities:', list(bg.keys()))
    
    rho_cdm = bg["rho_cdm"][:]
    rho_crit = bg["rho_crit"][:]
    rho_tot = bg["rho_tot"][:]
    rho_b = bg["rho_b"][:]
    rho_dcdm = bg["rho_dcdm"][:]
    rho_dr = bg["rho_dr"][:]
    rho_bcdm = bg["rho_b+cdm"][:]

    Odcdm = rho_dcdm / rho_crit
    Ocdm = rho_cdm / rho_crit
    Ob = rho_b / rho_crit
    Orad = rho_dr / rho_crit
    Obcdm = rho_bcdm / rho_crit

In [ ]:
# run this on all snapshot files for which you want to modify the background cosmology values with the "effective" values. 
# it will create a new modified snapshot file and leave the original one unchanged

# update these file names to the original snapshot and the name of the new modified effective snapshot file:
snapshot_og = "/mnt/c/Users/doman/OneDrive/Desktop/dcdmtut3/snapshot_a=0.60"
snapshot_eff = "/mnt/c/Users/doman/OneDrive/Desktop/dcdmtut3/snapshot_a=0.60_eff"

a_snap = 0.6 # edit this to be the scale factor of the snapshot being modified
# fill in the scale factor, early matter, late matter, and radiation (decay products) densities, and they need to be array-like
a = a_bg
em = rho_dcdm
lm = rho_cdm
rad = rho_dr
# include the true h0 of the simulation
h0 = 0.67


# print values of original snapshot file
print("Before:")
read_gadget_header_basic(snapshot_og)

patch_snapshot(
    snapshot_in=snapshot_og,     # original from CONCEPT
    snapshot_out=snapshot_eff, # patched for Rockstar usage
    a_snap=a_snap,  
    a_bg=a,
    rho_em= em,
    rho_lm= lm,
    rho_rad= rad,
    h0_true=h0
)

# print new values to double check the change
print("\nAfter:")
read_gadget_header_basic(snapshot_eff)
